<h1>CycleGAN for Style Transfer</h1>
<h3> Imports </h3>

In [11]:
#Python imports
import torch
import torch.nn as tnn
from torch.autograd import Variable
import sys
import itertools
from torch.utils.data import DataLoader
from Dataset import ImageSet
import datetime
import torchvision.utils as tv
import os
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image

#Import network elements
import network_elements as net

#stdout code blocks reload's print statement
#prevstd = sys.stdout
#sys.stdout = None
reload(net)
print("Reloading net")
#sys.stdout = prevstd

Reloading net


<h3>Set parameters</h3>

In [12]:
#The number of channels in the input image
im_in_channels = 3

#The number of channels in the output image
im_out_channels = 3

#The size of the largest size of the input image
im_size = 128

LEARNING_RATE = 0.0002

LR_DECAY_START = 100
LR_DECAY_END = 200
NUM_EPOCHS = 200

load_partial_net = False
CURR_EPOCH = 0

<h3>Import dataset</h3>

In [13]:
# apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos"
imageSet = ImageSet();
dataset = "monet2photo"
imageSet.downloadData(dataset)
training_transforms = [transforms.Resize(int(im_size*1.12), Image.BICUBIC),
                  transforms.RandomCrop(im_size),
                  transforms.RandomHorizontalFlip(),
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))];
imageSet.loadData(dataset, 'train', im_size, training_transforms)
imgLoader = DataLoader(imageSet,shuffle=True)

./datasets/monet2photo
Finished loading data


<h3>Define the CycleGAN network</h3>

In [14]:
#G : X->Y
G = net.Mapping(im_in_channels, im_out_channels, im_size)

#F : Y->X
F = net.Mapping(im_out_channels, im_out_channels, im_size)

#G mapping discriminator
Dx = net.Discriminator(im_in_channels)

#F mapping discriminator
Dy = net.Discriminator(im_out_channels)

#Initialize CUDA implementations
G.cuda();
F.cuda();
Dx.cuda();
Dy.cuda();

#Define losses
cycle_loss = tnn.L1Loss(); #Cycle-consistency loss
gan_loss = tnn.MSELoss(); #Adversarial loss
identity_loss = tnn.L1Loss(); #Prevents image tinting

<h3>Initialize weights for each network</h3>

In [15]:
# norm_weights function applies Gaussian norm weights with mean 0 and stddev 0.02
G.apply(net.conv_norm_weights);
F.apply(net.conv_norm_weights);
Dx.apply(net.conv_norm_weights);
Dy.apply(net.conv_norm_weights);

<h3>Set optimizer parameters</h3>

In [16]:
#simplified version has no pooling, paper does not, paper code does...
# REMOVE

gan_opt = torch.optim.Adam(itertools.chain(G.parameters(), F.parameters()),
                           lr = LEARNING_RATE)
dx_opt = torch.optim.Adam(Dx.parameters(), lr = LEARNING_RATE)
dy_opt = torch.optim.Adam(Dy.parameters(), lr = LEARNING_RATE)

The paper uses Adam optimizer with a learning rate of 0.0002 for the first 100 epochs that linearly decays to 0 over the next 100 epochs.

<h3> Initialize training variables </h3>

In [17]:
x = torch.cuda.FloatTensor(1, im_in_channels, im_size, im_size)
y = torch.cuda.FloatTensor(1, im_out_channels, im_size, im_size)

# Variable wrapper is depricated in >0.4
dReal = Variable(torch.cuda.FloatTensor(1).fill_(1.0), requires_grad = False)
dFake = Variable(torch.cuda.FloatTensor(1).fill_(0.0), requires_grad = False)

buffX_Y = net.MapBuffer()
buffY_X = net.MapBuffer()

if not os.path.isdir("./img/"):
    os.mkdir("./img/")

if not os.path.isdir("./img/" + dataset + "/"):
    os.mkdir("./img/" + dataset + '/')
    
if not os.path.isdir("./log/"):
    os.mkdir("./log")
    
if not os.path.isdir("./log/" + dataset + "/"):
    os.mkdir("./log/" + dataset + "/")
    
if os.path.isfile("./log/" + dataset + "/loss.log") and not load_partial_net:    
    os.remove("./log/" + dataset + "/loss.log")
    
logfile = open("./log/" + dataset + "/loss.log","a")

<h3> Load Partial Net </h3>

In [18]:
if (load_partial_net):
    if os.path.isfile("./model/" + dataset + "/model_info.txt"):
        model_file = open("./model/" + dataset + "/model_info.txt", "r")
        CURR_EPOCH = int(model_file.read())
        model_file.close()
    G.load_state_dict(torch.load('./model/' + dataset + '/G.data'))
    F.load_state_dict(torch.load('./model/' + dataset + '/F.data'))
    Dx.load_state_dict(torch.load('./model/' + dataset + '/Dx.data'))
    Dy.load_state_dict(torch.load('./model/' + dataset + '/Dy.data'))
    gan_opt.load_state_dict(torch.load('./model/' + dataset + '/gan_opt.data'))
    dx_opt.load_state_dict(torch.load('./model/' + dataset + '/dx_opt.data'))
    dy_opt.load_state_dict(torch.load('./model/' + dataset + '/dy_opt.data'))
    
last_epoch = -1
if (load_partial_net):
    last_epoch = CURR_EPOCH-1

gan_lr = torch.optim.lr_scheduler.LambdaLR(gan_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dx_lr = torch.optim.lr_scheduler.LambdaLR(dx_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dy_lr = torch.optim.lr_scheduler.LambdaLR(dy_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)

<h3> Training </h3>

In [ ]:
start_time = datetime.datetime.now()
for i in range(CURR_EPOCH,NUM_EPOCHS):
    for j, img in enumerate(imgLoader):
        elapsed_time = datetime.datetime.now()-start_time
        print("Epoch %d/%d, %d/%d, %s"%(i,LR_DECAY_END,j,len(imageSet),str(elapsed_time)))
        src_x = Variable(x.copy_(img['x']))
        src_y = Variable(y.copy_(img['y']))
        
        gan_opt.zero_grad()
        
        #identity Loss
        mapY_Y = G(src_y)
        lossY_Y = identity_loss(mapY_Y, src_y)*5.0
        
        mapX_X = F(src_x)
        lossX_X = identity_loss(mapX_X, src_x)*5.0
        
        #GAN Loss
        mapX_Y = G(src_x) #Generate image Y' from image X
        dX_Y = Dy(mapX_Y) #Check if image Y' seems real
        lossX_Y = gan_loss(dX_Y, dReal) #Calculate Y' realism error
        
        mapY_X = F(src_y) #Generate image X' from image Y
        dY_X = Dx(mapY_X) #Check if image X' seems real
        lossY_X = gan_loss(dY_X, dReal) #Calculate X' realism error
        
        #Cycle Consistency Loss
        mapX_Y_X = F(mapX_Y)
        lossX_Y_X = cycle_loss(mapX_Y_X, src_x)*10.0
        
        mapY_X_Y = G(mapY_X)
        lossY_X_Y = cycle_loss(mapY_X_Y, src_y)*10.0
        
        mapLoss = lossY_Y + lossX_X + lossX_Y + lossY_X + lossX_Y_X + lossY_X_Y
        mapLoss.backward()
        
        gan_opt.step()
        
        #Dx
        dx_opt.zero_grad()
        
        dX_res = Dx(src_x)
        lossX = gan_loss(dX_res, dReal)
        
        mapY_X_old = buffY_X.cycle(mapY_X)
        dY_X = Dx(mapY_X_old.detach())
        lossY_X = gan_loss(dY_X, dFake)
        
        dxLoss = (lossX + lossY_X)*0.5
        dxLoss.backward()
        
        dx_opt.step()
        
        #Dy
        dy_opt.zero_grad()
        
        dY_res = Dy(src_y)
        lossY = gan_loss(dY_res, dReal)
        
        mapX_Y_old = buffX_Y.cycle(mapX_Y)
        dX_Y = Dy(mapX_Y_old.detach())
        lossX_Y = gan_loss(dX_Y, dFake)
        
        dyLoss = (lossY + lossX_Y)*0.5
        dyLoss.backward()
        
        dy_opt.step()     
        
        print("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
        logfile.write("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
    tv.save_image(src_x.data, './img/' + dataset + '/src_x_%d.jpg'%(i))
    tv.save_image(src_y.data, './img/' + dataset + '/src_y_%d.jpg'%(i))
    tv.save_image(mapX_Y.data, './img/' + dataset + '/mapX_Y_%d.jpg'%(i))
    tv.save_image(mapY_X.data, './img/' + dataset + '/mapY_X_%d.jpg'%(i))
    gan_lr.step()
    dx_lr.step()
    dy_lr.step()
    

    if not os.path.isdir("./model/"):
        os.mkdir("./model/")
    
    if not os.path.isdir("./model/" + dataset + "/"):
        os.mkdir("./model/" + dataset + "/")
    model_file = open("./model/" + dataset + "/model_info.txt", "w")
    model_file.write("%d"%(i+1))
    model_file.close()
    torch.save(G.state_dict(), './model/' + dataset + '/G.data')
    torch.save(F.state_dict(), './model/' + dataset + '/F.data')
    torch.save(Dx.state_dict(), './model/' + dataset + '/Dx.data')
    torch.save(Dy.state_dict(), './model/' + dataset + '/Dy.data')
    torch.save(gan_opt.state_dict(), './model/' + dataset + '/gan_opt.data')
    torch.save(dx_opt.state_dict(), './model/' + dataset + '/dx_opt.data')
    torch.save(dy_opt.state_dict(), './model/' + dataset + '/dy_opt.data')

Epoch 0/200, 0/6287, 0:00:00.010941
0,0,22.244240,7.150340,0.932339,14.641977,0.650065

Epoch 0/200, 1/6287, 0:00:00.183461
0,1,20.154627,6.379827,0.443626,12.994964,0.675985

Epoch 0/200, 2/6287, 0:00:00.379804
0,2,17.737217,5.483084,0.139422,11.048169,0.553349

Epoch 0/200, 3/6287, 0:00:00.542682
0,3,15.726242,5.035377,0.554314,10.196634,0.407055

Epoch 0/200, 4/6287, 0:00:00.703805
0,4,14.072522,4.498518,1.035155,9.303916,0.701004

Epoch 0/200, 5/6287, 0:00:00.863404
0,5,16.381510,5.401195,1.648003,10.938687,0.997079

Epoch 0/200, 6/6287, 0:00:01.035803
0,6,13.635860,4.308844,0.417084,8.591006,0.579216

Epoch 0/200, 7/6287, 0:00:01.195947
0,7,13.327601,3.895491,0.043757,7.964617,1.152504

Epoch 0/200, 8/6287, 0:00:01.361947
0,8,14.118649,3.274919,0.474695,6.731483,1.601900

Epoch 0/200, 9/6287, 0:00:01.526512
0,9,13.253946,3.830964,0.060971,7.980385,0.734272

Epoch 0/200, 10/6287, 0:00:01.690002
0,10,14.569380,4.740540,1.359468,9.763831,0.873391

Epoch 0/200, 11/6287, 0:00:01.850332

0,93,7.509989,2.332299,0.614822,4.586262,0.442487

Epoch 0/200, 94/6287, 0:00:15.503205
0,94,10.305077,3.313448,0.381758,6.636001,0.587269

Epoch 0/200, 95/6287, 0:00:15.671740
0,95,9.497735,2.816250,0.295853,5.777036,0.266596

Epoch 0/200, 96/6287, 0:00:15.836786
0,96,13.834138,4.036611,0.337726,7.927858,0.173431

Epoch 0/200, 97/6287, 0:00:16.002895
0,97,8.049125,2.228316,0.479022,4.838796,0.379522

Epoch 0/200, 98/6287, 0:00:16.183871
0,98,11.969406,3.520796,0.076366,7.194936,0.101979

Epoch 0/200, 99/6287, 0:00:16.348894
0,99,8.657780,2.652150,1.220949,5.444381,1.018160

Epoch 0/200, 100/6287, 0:00:16.513776
0,100,8.120255,2.358566,0.312230,4.881160,0.220015

Epoch 0/200, 101/6287, 0:00:16.678643
0,101,7.344672,2.368087,0.107877,4.300972,0.121351

Epoch 0/200, 102/6287, 0:00:16.852194
0,102,8.125840,2.518026,1.356052,5.522623,0.839108

Epoch 0/200, 103/6287, 0:00:17.014221
0,103,9.427806,3.111698,2.250743,6.254789,1.166889

Epoch 0/200, 104/6287, 0:00:17.180752
0,104,7.179770,2.074

0,185,10.426163,3.087893,0.147751,6.520814,0.311501

Epoch 0/200, 186/6287, 0:00:30.840853
0,186,9.132163,2.489363,0.018019,5.583809,0.617489

Epoch 0/200, 187/6287, 0:00:31.005388
0,187,11.235228,3.924054,0.339356,6.980153,0.538792

Epoch 0/200, 188/6287, 0:00:31.176620
0,188,7.275659,2.429605,1.462863,4.543813,1.157192

Epoch 0/200, 189/6287, 0:00:31.354837
0,189,5.353257,1.787808,1.325986,3.467584,0.837262

Epoch 0/200, 190/6287, 0:00:31.521527
0,190,10.260607,3.360588,0.026072,6.090652,0.221612

Epoch 0/200, 191/6287, 0:00:31.682545
0,191,9.467627,2.533671,0.237633,5.413643,0.130284

Epoch 0/200, 192/6287, 0:00:31.844751
0,192,7.029444,1.683450,0.787732,3.642261,0.781776

Epoch 0/200, 193/6287, 0:00:32.008647
0,193,12.107812,3.583661,0.387323,8.195124,0.210174

Epoch 0/200, 194/6287, 0:00:32.170548
0,194,8.303360,2.166359,0.101411,5.099870,0.823114

Epoch 0/200, 195/6287, 0:00:32.334871
0,195,9.782505,3.017775,0.461850,6.199495,0.330459

Epoch 0/200, 196/6287, 0:00:32.501905
0,196,

0,277,10.756348,3.205274,0.668868,6.415462,0.380547

Epoch 0/200, 278/6287, 0:00:46.326953
0,278,8.562854,2.521517,0.298755,5.276774,0.236910

Epoch 0/200, 279/6287, 0:00:46.494973
0,279,11.168360,3.537354,0.352496,6.931797,0.234090

Epoch 0/200, 280/6287, 0:00:46.661864
0,280,9.191629,2.860198,0.362279,5.851110,0.498904

Epoch 0/200, 281/6287, 0:00:46.827262
0,281,8.521547,2.464407,0.339160,5.326242,0.510415

Epoch 0/200, 282/6287, 0:00:47.003030
0,282,8.428885,2.503540,0.088939,5.087104,0.078846

Epoch 0/200, 283/6287, 0:00:47.166109
0,283,6.795431,1.898485,0.686533,4.171607,0.668809

Epoch 0/200, 284/6287, 0:00:47.329750
0,284,7.579320,2.201488,0.250625,4.941345,0.307987

Epoch 0/200, 285/6287, 0:00:47.500075
0,285,8.143050,2.466046,0.457978,5.280463,0.418190

Epoch 0/200, 286/6287, 0:00:47.676715
0,286,8.320696,2.487019,0.870511,5.455118,0.554845

Epoch 0/200, 287/6287, 0:00:47.839383
0,287,10.521648,3.374689,0.760588,6.607272,0.631794

Epoch 0/200, 288/6287, 0:00:48.005461
0,288,8

0,369,12.106974,3.555627,1.018741,7.626310,0.647867

Epoch 0/200, 370/6287, 0:01:01.926526
0,370,12.688049,3.691383,0.240914,7.155460,0.432005

Epoch 0/200, 371/6287, 0:01:02.101018
0,371,6.121577,1.701490,0.362185,3.805117,0.293846

Epoch 0/200, 372/6287, 0:01:02.276527
0,372,10.275104,3.018952,0.237784,5.835248,0.583650

Epoch 0/200, 373/6287, 0:01:02.450353
0,373,14.241367,4.303297,0.047643,8.462687,0.240824

Epoch 0/200, 374/6287, 0:01:02.628008
0,374,8.038100,2.459695,0.473428,4.926450,0.392574

Epoch 0/200, 375/6287, 0:01:02.801037
0,375,6.606603,1.974374,0.369505,4.326834,0.317012

Epoch 0/200, 376/6287, 0:01:02.980334
0,376,9.594917,3.119297,0.067097,5.925956,0.116686

Epoch 0/200, 377/6287, 0:01:03.156943
0,377,10.085854,3.284325,0.595762,6.107644,0.320598

Epoch 0/200, 378/6287, 0:01:03.335673
0,378,10.855540,2.887792,0.427983,6.375628,0.217115

Epoch 0/200, 379/6287, 0:01:03.502617
0,379,10.655416,3.191865,0.146562,6.379290,0.273119

Epoch 0/200, 380/6287, 0:01:03.671073
0,3

...

Epoch 0/200, 1904/6287, 0:05:24.122248
0,1904,9.191916,2.422805,0.053010,5.172800,0.512406

Epoch 0/200, 1905/6287, 0:05:24.296148
0,1905,7.565437,1.892308,0.057023,3.935812,0.217660

Epoch 0/200, 1906/6287, 0:05:24.475457
0,1906,10.001408,2.203341,0.013650,5.341117,0.341858

Epoch 0/200, 1907/6287, 0:05:24.644941
0,1907,8.847935,2.323860,0.393568,5.413007,0.869406

Epoch 0/200, 1908/6287, 0:05:24.822254
0,1908,8.957253,2.424819,0.094922,5.290683,0.085587

Epoch 0/200, 1909/6287, 0:05:24.989707
0,1909,11.448380,3.427584,0.270770,7.019314,0.154394

Epoch 0/200, 1910/6287, 0:05:25.159878
0,1910,14.580348,4.060072,0.411768,10.046162,0.291278

Epoch 0/200, 1911/6287, 0:05:25.336179
0,1911,8.160654,2.211525,0.616026,4.855105,0.409086

Epoch 0/200, 1912/6287, 0:05:25.504685
0,1912,7.694075,2.236478,0.865795,5.123874,0.670476

Epoch 0/200, 1913/6287, 0:05:25.671986
0,1913,5.435093,1.490048,0.301254,3.377543,0.586770

Epoch 0/200, 1914/6287, 0:05:25.851776
0,1914,10.062141,2.814904,0.070769,5.

0,1993,5.963599,1.730069,0.146050,3.963957,0.115007

Epoch 0/200, 1994/6287, 0:05:39.438892
0,1994,7.156377,2.455386,0.332180,4.527145,0.228755

Epoch 0/200, 1995/6287, 0:05:39.599974
0,1995,11.775066,3.685045,0.295906,7.231339,0.243204

Epoch 0/200, 1996/6287, 0:05:39.772855
0,1996,7.183967,1.668751,0.270241,4.226642,0.277596

Epoch 0/200, 1997/6287, 0:05:39.937491
0,1997,10.748854,2.958449,0.132835,6.365805,0.080801

Epoch 0/200, 1998/6287, 0:05:40.101642
0,1998,7.056828,1.899606,0.035572,3.750041,0.369013

Epoch 0/200, 1999/6287, 0:05:40.265494
0,1999,6.224940,1.729609,0.121451,3.530281,0.314792

Epoch 0/200, 2000/6287, 0:05:40.430162
0,2000,9.534615,2.581766,0.202909,5.906517,0.226322

Epoch 0/200, 2001/6287, 0:05:40.595874
0,2001,7.484361,2.250419,0.128491,4.237279,0.503582

Epoch 0/200, 2002/6287, 0:05:40.762984
0,2002,6.726722,1.619345,0.207896,2.948946,0.694965

Epoch 0/200, 2003/6287, 0:05:40.929293
0,2003,13.248135,3.738540,0.445916,7.667911,0.383144

Epoch 0/200, 2004/6287, 

0,2082,8.827540,2.515414,0.545266,5.678504,0.370498

Epoch 0/200, 2083/6287, 0:05:54.476190
0,2083,9.180479,2.365874,0.123122,5.596650,0.246671

Epoch 0/200, 2084/6287, 0:05:54.642514
0,2084,6.850434,2.030701,0.372863,4.173362,0.336026

Epoch 0/200, 2085/6287, 0:05:54.817191
0,2085,7.511168,2.438530,0.074200,4.379641,0.142049

Epoch 0/200, 2086/6287, 0:05:54.984291
0,2086,6.548649,1.958963,0.298809,4.003896,0.549799

Epoch 0/200, 2087/6287, 0:05:55.162739
0,2087,6.830312,1.922394,0.354903,4.431469,0.352063

Epoch 0/200, 2088/6287, 0:05:55.336635
0,2088,4.449808,1.461022,0.831646,2.694633,0.528857

Epoch 0/200, 2089/6287, 0:05:55.501027
0,2089,8.801099,2.571254,0.207913,5.339154,0.140466

Epoch 0/200, 2090/6287, 0:05:55.665303
0,2090,7.003242,2.031548,0.361300,4.096142,0.514946

Epoch 0/200, 2091/6287, 0:05:55.829075
0,2091,8.441812,2.677393,0.516771,5.077122,0.464235

Epoch 0/200, 2092/6287, 0:05:55.993419
0,2092,6.804101,1.864329,0.242042,4.049974,0.329578

Epoch 0/200, 2093/6287, 0:0

0,2172,7.870696,2.440676,0.315788,4.932544,0.319903

Epoch 0/200, 2173/6287, 0:06:09.984246
0,2173,7.842772,2.119821,0.284252,4.996888,0.331077

Epoch 0/200, 2174/6287, 0:06:10.150305
0,2174,6.334914,2.009399,0.352140,3.613550,0.312356

Epoch 0/200, 2175/6287, 0:06:10.317723
0,2175,6.241074,1.823155,0.198158,3.346125,0.417733

Epoch 0/200, 2176/6287, 0:06:10.482019
0,2176,8.529482,2.720743,0.644913,5.074611,0.418750

Epoch 0/200, 2177/6287, 0:06:10.650129
0,2177,9.095065,2.807914,0.639777,5.909708,0.413128

Epoch 0/200, 2178/6287, 0:06:10.819057
0,2178,7.081415,2.221230,0.357566,4.046690,0.272214

Epoch 0/200, 2179/6287, 0:06:11.001303
0,2179,6.764416,2.100927,0.281802,3.761409,0.202324

Epoch 0/200, 2180/6287, 0:06:11.175576
0,2180,7.705219,2.063877,0.051253,4.232903,0.273789

Epoch 0/200, 2181/6287, 0:06:11.342104
0,2181,10.105090,3.151951,0.300896,6.383983,0.661750

Epoch 0/200, 2182/6287, 0:06:11.508362
0,2182,5.079526,1.340815,0.272236,2.939558,0.222846

Epoch 0/200, 2183/6287, 0:

0,2262,6.604745,1.876313,0.340465,4.039958,0.305865

Epoch 0/200, 2263/6287, 0:06:25.597462
0,2263,7.847116,2.508362,0.461982,5.200217,0.539402

Epoch 0/200, 2264/6287, 0:06:25.774265
0,2264,8.241440,2.602883,0.671679,5.269098,0.366097

Epoch 0/200, 2265/6287, 0:06:25.956144
0,2265,8.449683,2.757350,0.032339,5.556087,0.045376

Epoch 0/200, 2266/6287, 0:06:26.143259
0,2266,4.801265,1.353290,0.858146,3.015000,0.443444

Epoch 0/200, 2267/6287, 0:06:26.316708
0,2267,6.471489,1.811617,0.197180,3.601923,0.189146

Epoch 0/200, 2268/6287, 0:06:26.496348
0,2268,7.223846,2.352361,0.242974,4.152226,0.182683

Epoch 0/200, 2269/6287, 0:06:26.672049
0,2269,15.266252,3.996608,0.131538,10.125533,0.212564

Epoch 0/200, 2270/6287, 0:06:26.841462
0,2270,10.040293,2.685197,0.290865,5.957933,0.297652

Epoch 0/200, 2271/6287, 0:06:27.009164
0,2271,9.832395,2.821450,0.091986,5.775933,0.242415

Epoch 0/200, 2272/6287, 0:06:27.185181
0,2272,5.872867,1.488648,0.145727,3.486780,0.534807

Epoch 0/200, 2273/6287, 

In [ ]:
logfile.close()

In [ ]:
print("Finished training")